Title

Link to the dataset: https://www.kaggle.com/datasets/xtrnglc/spotify-mbti-playlists

Introduction:

* Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
* Clearly state the question you will try to answer with your project
* Identify and describe the dataset that will be used to answer the question

Preliminary exploratory data analysis:
* Demonstrate that the dataset can be read from the web into R 
* Clean and wrangle your data into a tidy format
* Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
* Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

Methods:
* Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
* Describe at least one way that you will visualize the results

Expected outcomes and significance:

* What do you expect to find?

It is difficult to predict with confidence what we will find by analyzing music types and MBTI because music preference is complex and highly unique. However, we may be able to find correlations between certain MBTI types and specific factors in music, such as energy levels, danceability, key, loudness, etc. Using these trends we hope to predict an individual’s MBTI type based on variables of the music they enjoy.

* What impact could such findings have?

Although the MBTI system is a controversial topic in the psychology community due to its inability to fully represent the intricacies of human personality, our project may prove to be useful in developing a general understanding of the relationship between music and personality. This knowledge may be purposeful in many areas, including but not limited to:

Personal development: 
Individuals may utilize a better understanding of the relationship between their personality and music types to make well-informed decisions on their music of choice, or explore new types of music that may better align with their personality type.

Advertisement: 
A better understanding of the relationship between personality and music type may allow companies to better curate targeted marketing strategies, such as personalized advertisements for musical merchandise or specially-made playlists.

Music recommendation algorithms: 
Streaming platforms may be able to follow personality and music trends to suggest music more suitable to the listener’s personality type.

* What future questions could this lead to?
There are several questions that our project could lead to, such as:

Can musical preferences be used to suggest personality traits beyond MBTI types?

Is there a difference in musical preference depending on mood/situation?

How do environmental factors influence the relationship between music preferences and personality types?

